# ADAptive LInear NEuron (ADALINE)

### ¿Por qué son diferentes el perceptrón y el adaline?
<center><img src="https://drive.google.com/uc?export=view&id=1cqn7bXOEfDm7apLiNLzdUauMuaEW28Yb" width="880" alt="centered image"></center>
De la figura se puede ver que en el algoritmo del Adaline se comparan las etiquetas de las clases esperadas con el valor (continuo o número real) de salida de la función de activación $\phi$, de esta diferencia se calcula el error para actualizar los pesos. 
En el perceptrón, por el contrario, se comparan las etiquetas de las clases esperadas con las etiquetas obtenidas en la predicción.

### ¿Qué es la función de costo $J(w)$?
Es la función que se busca minimizar durante el proceso de aprendizaje o entrenamiento.
En el caso del Adaline, podemos definirla como la _suma de los errores cuadráticos_ entre la salida calculada (con la fn. de activación $\phi(y)$) y la etiqueta de clase esperada $y_d$. 
$$J(w) = \frac{1}{2}\sum\limits_{i=1}^N(y_d^{(i)} - \phi(y^{(i)}))^2$$
Para el Adaline vamos a utilizar un algoritmo de optimización llamado _Descenso del Gradiente_.

En este caso, la actualización de los pesos se hace de la siguiente manera: $$\vec{w} = \vec{w} + \Delta\vec{w}$$
Donde $\Delta{w}$ queda definido como el gradiente negativo de la fn. de costo multiplicado por el factor de entrenamiento $$\Delta{\vec{w}} = - \mu\nabla{J}(\vec{w})$$


### Pero...¿Cómo calculamos el gradiente de la fn. de costo?
Una propiedad de nuestra función de costo es que es derivable, a diferencia de la fn. signo del perceptrón. Por tanto sólo debemos calcular la derivada parcial de la función de costo respecto de cada peso $w_j$ $$J(w) = \frac{1}{2}\sum\limits_{i=1}^N(y_d^{(i)} - \phi(y^{(i)}))^2$$ 

$$\frac{\partial{J}}{\partial{w_j}} = - \sum\limits_{i}(y^{(i)} - \phi(z^{(i)}))x^{(i)}_j$$ y la actualización de los pesos:
$$\Delta{w_j} = -\mu\frac{\partial{J}}{\partial{w_j}}=\mu\sum\limits_{i}(y^{(i)} - \phi(z^{(i)}))x^{(i)}_j$$

> La función de costo se calcula teniendo en cuenta todos los patrones de entrada (note el índice i en la ecuación de $J(w)$) por lo tanto la actualización de los pesos en el algoritmo del Adaline se calcula en base a todas las entradas (a diferencia de la actualización incremental de los pesos con cada patrón de entrada en el perceptrón). A esto se le llama también _Descenso del gradiente en lotes_ (batch gradient descent)

In [ ]:
import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class Adaline(object):
    """ADAptive LInear NEuron classifier.
    Parametros
    ------------
    lr : float
        factor de entrenamiento (entre 0.0 y 1.0)
    n_iter : int
        iteraciones para el entrenamiento.
    random_state : int
        Semilla generadora de números aleatorios para
        la inicialización de los pesos.
    
    Atributos
    -----------
    w_ : 1d-array
        Pesos despues de su ajuste.
    costo_ : list
        Función de costo : suma de los errores cuadráticos en cada época
    """
    def __init__(self, lr=0.01, n_iter=50, random_state=1):
        self.lr = lr
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """Ajuste de los datos de entrenamiento.
        
        Parameters
        ----------
        X : {array-like}, shape = [n_muestras, n_caracteristicas]
          vector de entrenamiento
        y : array-like, shape = [n_muestras]
          vector target.
          
        Returns
        -------
        self : objeto

        """
        rgen = RandomState(self.random_state)
        #inicializo los pesos con valores aleatorios entre 0 y 1
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.costo_ = []

        for _ in range(self.n_iter):
            entrada = self.calcular_entrada(X)
            salida = self.fn_activacion(entrada)
            # El error se calcula como la diferencia entre las salidas esperadas y
            # la salida de la fn. de activación
            errores = (y - salida)              
            # --------actualización del vector de pesos--------------------------------
            self.w_[1:] += self.lr * X.T.dot(errores)
            self.w_[0] += self.lr * errores.sum()
            # función de costo
            costo = (errores**2).sum() / 2.0
            #---------------------------------------------------------------------------
            self.costo_.append(costo)
        return self
    
    def calcular_entrada(self, X):
        """Cálculo de la entrada al Adaline"""
        # -------suma de los productos de los valores de entrada y los pesos -----------        
        return np.dot(X, self.w_[1:]) + self.w_[0]
        #-------------------------------------------------------------------------------
    
    def fn_activacion(self, X): 
        """Aplica la fn. de activación lineal"""
        #Se puede ver que la fn.  de activación es la fn. identidad, vamos a ver que en el
        #caso de la regresión logística (más adelante), usamos una fn. sigmoidea
        return X

    def predict(self, X):
        """devuelve la etiqueta de la clase pertenciente después de aplicar la fn. de activación"""
        return np.where(self.calcular_entrada(X) >= 0.0, 1, -1)

In [ ]:
#Cargo el Dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
y = df.iloc[0:100, 4].values
#convertimos las etiquetas de clases en  0 (Iris-setosa)  y 1 (Iris-versicolor)
y = np.where(y == 'Iris-setosa', -1, 1)
# columna 0 : longitud de sépalo, columna 2 : longitud de pétalo
X = df.iloc[0:100, [0, 2]].values
print(X.shape) #(100,2)
print(y.shape) #(100,)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)                     
X_std = sc.transform(X)

En la práctica, seleccionar el coeficiente de aprendizaje ($\mu$) para la convergencia del algoritmo requiere algo de experimentación. El valor de $\mu$ que seleccionemos hará la diferencia entre los dos casos que tenemos a continuación
<center><img src="https://drive.google.com/uc?export=view&id=1c-uve0PzNx9GQZvbBkxYf9arKatB0Rs5" width= 700 alt="centered image"></center>

Como podemos ver en la gráfica a la derecha, un valor de $\mu$ muy grande hará que el algoritmo del descenso del gradiente no converja y por lo tanto no se minimiza la función de costo $J(w)$. En la gráfica de la izquierda, un valor de $\mu$ muy chico hará que el algoritmo converja pero necesitará de un gran número de épocas para la convergencia.

### Vamos a crear dos instancias de la Clase Adaline, uno con lr = 0.1 y el otro con lr= 0.01 y grafiquemos sus funciones de costo!!! 

In [ ]:
ada1 = Adaline(n_iter=30, lr=0.1, random_state=1)
ada1.fit(X_std, y)
ada2 = Adaline(n_iter=30, lr=0.01, random_state=1)
ada2.fit(X_std, y)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
ax[0].plot( range(1, len(ada1.costo_) + 1), np.log10(ada1.costo_), marker='o')
ax[0].set_xlabel('Épocas')
ax[0].set_ylabel('log(Suma-error-cuadrático)')
ax[0].set_title('Adaline  lr = 0.1')

ax[1].plot( range(1, len(ada2.costo_) + 1), np.log10(ada2.costo_), marker='o')
ax[1].set_xlabel('Épocas')
ax[1].set_ylabel('Suma-error-cuadrático')
_ = ax[1].set_title('Adaline  lr = 0.01')

El algoritmo del adaline ilustra los conceptos claves de la definición y minimización de las funciones de costo continuas. Esto brinda las bases para el entendimiento de algoritmos de machine learning para clasificación más avanzados, como la regresión logística, máquinas de soporte vectorial y modelos de regresión que veremos más adelante en la asignatura.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, clasificador, test_idx=None, resolution=0.02):
    
    # setup marker generator and color map
    markers = ('s', 'o', '^', 'v', 'x')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    
    Z = clasificador.predict(np.array([xx1.ravel(), xx2.ravel()]).T)

    Z = Z.reshape(xx1.shape)

    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

    # highlight test samples
    if test_idx:
        # plot all samples
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c="None",
                    edgecolor='black',
                    alpha=1,
                    linewidth=1,
                    marker='o',
                    s=100, 
                    label='test set')

In [ ]:
plt.figure(figsize=(10,7))
plot_decision_regions(X_std,y, ada2)
plt.xlabel('longitud de pétalo')
plt.ylabel('ancho de pétalo ')
#plt.xlim([4, 7.2])
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()